In [237]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy, resample_apply
from backtesting.test import SMA
from backtesting import Backtest

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd 
import math as m
import os, glob

In [238]:
import warnings
warnings.filterwarnings("ignore")

In [239]:
PRED = True
WithClassOne = False
CASH = 1000
COMMISSION = 0.0001

In [240]:
pwd = os.getcwd()
datasource_path = os.path.join(pwd, r'../../../Results/LSTM/')
filenames = glob.glob(datasource_path + "/*.csv")
filenames.sort()
for file in filenames:
    print(os.path.basename(file))

3C-1H-LSTM_1H_43F_Class_01_Test.csv
3C-1H-LSTM_1H_43F_Class_01_Train.csv
5C-1D-LSTM_1D_43F_Class_01_Test.csv
5C-1D-LSTM_1D_43F_Class_01_Train.csv
5C-LSTM_1H_42F_Class_01_Test.csv
5C-LSTM_1H_42F_Class_01_Train.csv
C-LSTM_1D_19F_Class_03_Test.csv
C-LSTM_1D_19F_Class_03_Train.csv
C-LSTM_1H_0F_Class_Test_Results_01.csv
C-LSTM_1H_0F_Class_Test_Results_02.csv
C-LSTM_1H_0F_Class_Train_Results_01.csv
C-LSTM_1H_0F_Class_Train_Results_02.csv


In [241]:
index = 3

dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
DATASET = pd.read_csv(filenames[index], header=0)
MODEL_NAME = os.path.basename(filenames[index])
print(MODEL_NAME)
print(DATASET.head(20))
print(DATASET.shape)


5C-1D-LSTM_1D_43F_Class_01_Train.csv
       Open     High      Low    Close        Volume  Real_Class  \
0   1.21846  1.22227  1.21573  1.21868  1.061680e+06        -1.0   
1   1.21819  1.21847  1.21594  1.21664  7.823640e+04         0.0   
2   1.21663  1.21796  1.21227  1.21730  1.110667e+06         2.0   
3   1.21715  1.23465  1.21675  1.23245  1.111821e+06        -2.0   
4   1.23252  1.23314  1.22287  1.22667  1.113433e+06         1.0   
5   1.22680  1.23330  1.22526  1.22766  1.113054e+06         0.0   
6   1.22761  1.23610  1.22378  1.22740  1.067122e+06         0.0   
7   1.22812  1.22868  1.22662  1.22761  7.687460e+04         1.0   
8   1.22741  1.23156  1.22465  1.23009  1.119237e+06         1.0   
9   1.23002  1.23472  1.22778  1.23138  1.117918e+06         1.0   
10  1.23134  1.23401  1.22884  1.23281  1.125524e+06         2.0   
11  1.23293  1.24437  1.23141  1.24271  1.120084e+06        -1.0   
12  1.24249  1.24343  1.23837  1.23998  1.067855e+06        -1.0   
13  1.23949

In [242]:
NumCLASS =  DATASET['Predicted_Class'].value_counts().shape[0]
print(NumCLASS)

5


In [243]:
if NumCLASS == 3:
    if (PRED):
        DATASET['Position'] = DATASET['Predicted_Class']
        DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = -1 
        DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 1 
    else:
        DATASET['Position'] = DATASET['Real_Class']
        DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = -1
        DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 1 
if NumCLASS == 5:
    if (PRED):
        DATASET['Position'] = DATASET['Predicted_Class']
        if (WithClassOne):
            DATASET.loc[DATASET.Predicted_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Predicted_Class == 2, 'Position'] = 1
        else:
            DATASET.loc[DATASET.Predicted_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Predicted_Class == 2, 'Position'] = 1
            DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = 0 # remove Small movements
            DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 0 #Remove Small movements
    else:
        DATASET['Position'] = DATASET['Real_Class']
        if (WithClassOne):
            DATASET.loc[DATASET.Real_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Real_Class == 2, 'Position'] = 1
        else:
            DATASET.loc[DATASET.Real_Class == -2, 'Position'] = -1
            DATASET.loc[DATASET.Real_Class == 2, 'Position'] = 1
            DATASET.loc[DATASET.Predicted_Class == -1, 'Position'] = 0 # remove Small movements
            DATASET.loc[DATASET.Predicted_Class == 1, 'Position'] = 0 #Remove Small movements
DATASET.head(10)

,Open,High,Low,Close,Volume,Real_Class,Predicted_Class,Position
0,1.21846,1.22227,1.21573,1.21868,1.061680e+06,-1.0,0.0,0.0
1,1.21819,1.21847,1.21594,1.21664,7.823640e+04,0.0,1.0,0.0
2,1.21663,1.21796,1.21227,1.21730,1.110667e+06,2.0,-1.0,0.0
3,1.21715,1.23465,1.21675,1.23245,1.111821e+06,-2.0,-1.0,0.0
4,1.23252,1.23314,1.22287,1.22667,1.113433e+06,1.0,-1.0,0.0
5,1.22680,1.23330,1.22526,1.22766,1.113054e+06,0.0,2.0,1.0
6,1.22761,1.23610,1.22378,1.22740,1.067122e+06,0.0,0.0,0.0
7,1.22812,1.22868,1.22662,1.22761,7.687460e+04,1.0,1.0,0.0
8,1.22741,1.23156,1.22465,1.23009,1.119237e+06,1.0,2.0,1.0
9,1.23002,1.23472,1.22778,1.23138,1.117918e+06,1.0,-1.0,0.0


In [244]:
class limits(SignalStrategy, TrailingStrategy):
    buy_lim = 0.99
    sell_lim = -0.99
    
    def init(self):
        super().init()
        self.Op_Abierta = False
        
    def next(self):
        sig = self.data.Position[-1]
        if not self.Op_Abierta and sig >= self.buy_lim:
            self.buy()
            self.Op_Abierta = True
        if self.Op_Abierta and sig <= self.sell_lim:
            self.sell()
            self.Op_Abierta = False

In [245]:
class limits2(SignalStrategy, TrailingStrategy):
    buy_lim = 0.99
    sell_lim = -0.99
    
    def init(self):
        super().init()
        
    def next(self):
        sig = self.data.Position[-1]
        if  sig >= self.buy_lim:
            self.buy()
        if  sig <= self.sell_lim:
            self.sell()

In [246]:
class Position(SignalStrategy):
    def init(self):
        global df
        super().init()  
        self.set_signal(self.data.Position)

In [247]:
DATASET['Predicted_Class'].value_counts()

-2.0    973
 2.0    770
 1.0    643
 0.0    621
-1.0    565
Name: Predicted_Class, dtype: int64

In [248]:
DATASET['Real_Class'].value_counts()

 2.0    795
-2.0    787
-1.0    697
 1.0    687
 0.0    606
Name: Real_Class, dtype: int64

In [249]:
STRATEGY_NAME = limits
print(STRATEGY_NAME.__name__)

limits


In [250]:
bt = Backtest(DATASET, STRATEGY_NAME, cash=CASH, commission=COMMISSION)#trade_on_close=False
bt.run()

Start                             0
End                            3571
Duration                       3571
Exposure [%]                 99.776
Equity Final [$]            11438.7
Equity Peak [$]             11438.7
Return [%]                  1043.87
Buy & Hold Return [%]       8.29012
Max. Drawdown [%]          -26.5988
Avg. Drawdown [%]          -1.43103
Max. Drawdown Duration          836
Avg. Drawdown Duration      18.5795
# Trades                        527
Win Rate [%]                64.7059
Best Trade [%]               14.219
Worst Trade [%]             -7.4316
Avg. Trade [%]             0.468911
Max. Trade Duration             150
Avg. Trade Duration         6.76091
Expectancy [%]              1.14807
SQN                         4.41932
Sharpe Ratio               0.279473
Sortino Ratio              0.467482
Calmar Ratio              0.0176291
_strategy                    limits
dtype: object

In [251]:
bt.plot()

In [252]:
data = {'Model_Name':[MODEL_NAME], 'PRED':[PRED], 'Strategy_Class':[STRATEGY_NAME.__name__], 'Equity_Final_$':[bt._results['Equity Final [$]']],
       'Equity_Peak_$':[bt._results['Equity Peak [$]']],'Return_%':[bt._results['Return [%]']],'#_Trades':[bt._results['# Trades']],
       'Win_Rate':[bt._results['Win Rate [%]']],'Commission':[COMMISSION],'Cash':[CASH]}
Results = pd.DataFrame(data)
print(Results)

                             Model_Name  PRED Strategy_Class  Equity_Final_$  \
0  5C-1D-LSTM_1D_43F_Class_01_Train.csv  True         limits    11438.672948   

   Equity_Peak_$     Return_%  #_Trades   Win_Rate  Commission  Cash  
0   11438.672948  1043.867295     527.0  64.705882      0.0001  1000  


In [253]:
filename = 'BackTesting_Results.csv'
print(filename)
pwd = os.getcwd()
datasource_path = os.path.join(pwd, r'../../../Results/Backtesting/')
filename_path = os.path.join(pwd, r'../../../Results/Backtesting/' + filename)
print(filename_path)

BackTesting_Results.csv
C:\Users\maw\Documents\ITESO\forex_prediction\Jupyter Notebooks\IDI4\backtesting\../../../Results/Backtesting/BackTesting_Results.csv


In [254]:
#LOAD RESULTS .CSV
pwd = os.getcwd()
filenames = glob.glob(datasource_path + "/*.csv")
filenames.sort()
for file in filenames:
    print(os.path.basename(file))

BackTesting_Results.csv


In [255]:
Results_loaded = pd.read_csv(filenames[0], header=0)

In [256]:
#print(Results)
Results_loaded = Results_loaded.append(Results,ignore_index=True)


In [257]:
Results_loaded.to_csv(filename_path, header=True, index=False)